In [4]:
import contextlib
import itertools
import sqlite3
from pathlib import Path

import exiftool
from tqdm.auto import tqdm

In [5]:
data_root = Path(r"\\e4e-nas.ucsd.edu\fishsense_data\REEF\data")
data_db = Path(r"\\e4e-nas.ucsd.edu\fishsense_data\REEF\processing.db")
exiftool_path = Path(r"C:\Users\nthui\Downloads\exiftool-13.22_64\exiftool(-k).exe")

In [ ]:
with contextlib.closing(sqlite3.connect(data_db)) as con, contextlib.closing(con.cursor()) as cur:
    images = [data_root.joinpath(row[0]) for row in cur.execute('SELECT path FROM images WHERE images.camera_sn ISNULL;').fetchall()]

    with exiftool.ExifToolHelper(executable=exiftool_path.as_posix()) as et:
        for image_batch in itertools.batched(tqdm(images, 'Image Camera SN'), n=1):
            try:
                metadata = et.get_tags(image_batch, ['MakerNotes:SerialNumber'])
            except Exception as exc:
                print(exc)
                continue

            for data in metadata:
                try:
                    params = {
                                    'camera_sn': data['MakerNotes:SerialNumber'].strip(),
                                    'path': Path(data['SourceFile']).relative_to(data_root).as_posix()
                                }
                except KeyError:
                    continue
                cur.execute('UPDATE images SET camera_sn = :camera_sn WHERE images.path = :path',
                            params)
                con.commit()

Image Camera SN: 100%|██████████| 55/55 [00:00<00:00, 155.29it/s]

execute returned a non-zero exit status: 1
